In [1]:
import os
import sys

PROJECT_ROOT = os.path.join(os.path.abspath(os.path.dirname('./../../')), 'src')

sys.path.append(PROJECT_ROOT)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [4]:
conf = {'groups_desc': 'data/small_group_description.csv',
    'train_x': 'data/transactions_train.csv',
    'test_x': 'data/transactions_test.csv',
    'train_target': 'data/train_target.csv',
    'test_target': 'data/test.csv'}

In [5]:
from data_utils import *

In [6]:
%%time

ds = DataSource(conf)
ds.read_data()
ds.validation_split()

Reading groups_desc...
Reading train_x...
Reading test_x...
Reading train_target...
Reading test_target...
Validation split: by clientID
Wall time: 24.7 s


In [83]:
ds.get_data('groups_desc')

,small_group,small_group_code
0,Зоотовары,0
1,Сетевые супермаркеты и продуктовые магазины,1
2,Обслуживание существующего автомобиля (АЗС),2
3,Оплата телефона и связи,3
4,Аптеки,4
5,Авиабилеты,5
6,Duty-free,6
7,Страховка,7
8,Спорт,8
9,Универсальные товары,9


In [8]:
train_x = ds.get_data('train_x')
print(train_x.head())


   client_id  trans_date  small_group  amount_rur
0       3694          26            1      10.006
1       3694          29            3      54.955
2       3694          31            1      10.945
3       3694          34           36      61.721
4       3694          36           25       4.579


In [60]:
train_x['month_number'] = train_x['trans_date']//30

In [33]:
train_x.head()

,client_id,trans_date,small_group,amount_rur,month_number
0,3694,26,1,10.006,0
1,3694,29,3,54.955,0
2,3694,31,1,10.945,1
3,3694,34,36,61.721,1
4,3694,36,25,4.579,1


In [45]:
grouped = train_x.groupby(['client_id', 'month_number'], as_index=False)

In [50]:

client_per_month_spendings = train_x.groupby(['client_id', 'month_number']).agg({"amount_rur": "sum"}).groupby('client_id').agg({'amount_rur':'mean'})

In [60]:
print(train_x.groupby(['client_id']).size())


client_id
4         720
6         730
10        706
11        812
13        754
14       1097
17       1053
19        721
20        821
25        996
27        811
28        765
31        834
32       1035
33        908
34       1033
36        819
37        772
38        720
39        760
40        916
42        792
43        917
44        714
46        866
47        749
48        920
49        868
50        970
51        791
         ... 
49949     732
49950    1001
49951     915
49955     856
49957     870
49959     963
49961    1083
49962     829
49963     747
49967     702
49968     775
49971     704
49972    1082
49973     904
49974    1129
49975    1030
49977     721
49978     960
49980    1023
49981     757
49982     957
49983     904
49987    1120
49988     777
49989     706
49990     721
49995     969
49996    1138
49997     715
49998     782
Length: 24145, dtype: int64


In [10]:
grouped_by_group = train_x.groupby(['client_id', 'small_group'], as_index = False)['small_group'].agg({'freq':'count'})
client_per_max_frequent_category  = grouped_by_group.loc[grouped_by_group.groupby('client_id')['freq'].idxmax()][['client_id','small_group']]
client_per_max_frequent_category.columns = ['client_id', 'freq_group']
print(client_per_max_frequent_category.head())


     client_id  freq_group
0            4           1
23           6           1
40          10           1
68          11           1
108         13          11


In [49]:
# Вычислием среднюю максимальную транзакцию по клиентам. Далее найдем среднее отклонение максим транзакций по клиентам. 
# Это позволит найти "большие" транзакции , как mean + std. у нас не гаусово распределение, но все же оперделим значение "большой" транзакции 
# именно так.
grouped_by_max = train_x.groupby(['client_id'], as_index = False).agg({'amount_rur':'max'})
print(grouped_by_max.agg({'amount_rur':'mean'})['amount_rur'] )
large_transactions = grouped_by_max.agg({'amount_rur':'mean'})['amount_rur'] +grouped_by_max.agg({'amount_rur':'std'})['amount_rur']/40
print(large_transactions)

2765.900111078898
2941.7739788361896


In [50]:
len(train_x[train_x['amount_rur'] > large_transactions]['client_id'].unique())

5424

In [39]:
grouped_by_min = train_x.groupby(['client_id'], as_index = False).agg({'amount_rur':'min'})
small_transactions = grouped_by_min.agg({'amount_rur':'mean'})['amount_rur'] - grouped_by_min.agg({'amount_rur':'std'})['amount_rur']/2
print(small_transactions)

0.1025517531066103


In [44]:
len(train_x[train_x['amount_rur'] < small_transactions]['client_id'].unique())

13078

In [52]:
train_x[train_x['amount_rur'] < small_transactions].groupby(['client_id'], as_index = False)['trans_date'].agg({'small_num':'count'}).head()

,client_id,large_num
0,4,1
1,6,1
2,10,1
3,13,2
4,14,2


In [57]:
train_x[train_x['amount_rur'] > large_transactions].groupby(['client_id'], as_index = False)['trans_date'].agg({'large_num':'count'}).head()

,client_id,large_num
0,17,1
1,20,2
2,31,1
3,39,2
4,49,12


In [61]:

train_x.groupby(['client_id', 'month_number'])['amount_rur'].agg({"c": "count"}).groupby('client_id').agg({'c':'mean'})

,c
client_id,
4,28.800000
6,29.200000
10,28.240000
11,32.480000
13,30.160000
14,45.708333
17,42.120000
19,28.840000
20,32.840000


In [63]:
train_x[train_x['client_id'] == 4]

,client_id,trans_date,small_group,amount_rur,month_number
10064401,4,0,1,10.209,0
10064402,4,2,3,27.706,0
10064403,4,3,1,13.024,0
10064404,4,4,1,17.632,0
10064405,4,6,4,18.089,0
10064406,4,6,1,27.067,0
10064407,4,8,1,21.952,0
10064408,4,9,36,29.327,0
10064409,4,9,1,11.083,0
10064410,4,10,15,52.985,0
